1. Take plaintext as string
2. split it into 16 bytes
3. for each byte xor it with a key and sbox it
4. Calculate the hamming weight of the sbox output
4. save that data to a matrix 
5. sum the rows in the matrix 

In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("TkAgg")

In [111]:

def read_plaintext_line(filepath, line_number):
    """
    Reads a specific line from the plaintexts.txt file and splits it into 16 bytes.
    
    Args:
        filepath (str): Path to the plaintexts.txt file.
        line_number (int): Line number to read (0-indexed).
    
    Returns:
        list: A list of 16 integers representing the bytes of the plaintext.
    """
    with open(filepath, "r") as file:
        lines = file.readlines()
        if line_number < 0 or line_number >= len(lines):
            raise ValueError("Invalid line number.")
        line = lines[line_number].strip()
        return [int(line[i:i+2], 16) for i in range(0, len(line), 2)]

# Example usage:
# plaintext_bytes = read_plaintext_line("/Users/ashollerbach/Documents/GitHub/DPA/plaintexts.txt", 0)
# print(plaintext_bytes)



In [112]:
row = 501
plaintext_bytes = read_plaintext_line("/Users/ashollerbach/Documents/GitHub/DPA/plaintexts.txt", row)
print(plaintext_bytes)



[242, 34, 65, 171, 187, 35, 211, 141, 204, 92, 157, 155, 29, 156, 77, 243]


In [113]:
def hamming_weight(x):
    return bin(x).count('1')

In [114]:
keys = range(0, 256) # generate possible keys

In [115]:
sbox = np.array([
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
], dtype=np.uint8)

In [116]:
# Initialize 2D list to store Hamming weights
hamming_matrix = []

# For each key guess (0 to 255)
for keyguess in keys:
    # Compute Hamming weights for S-box output of plaintext XOR keyguess
    hamming_row = [hamming_weight(sbox[p ^ keyguess]) for p in plaintext_bytes]
    # Append row to 2D list
    hamming_matrix.append(hamming_row)

In [117]:
for row in hamming_matrix:
    print(row)

[3, 4, 3, 3, 5, 3, 4, 5, 4, 3, 5, 2, 3, 6, 5, 3]
[3, 3, 2, 4, 5, 4, 5, 3, 6, 3, 6, 4, 4, 5, 3, 3]
[3, 6, 3, 5, 4, 7, 5, 5, 4, 3, 6, 6, 2, 3, 2, 3]
[3, 7, 3, 3, 4, 6, 3, 3, 3, 6, 3, 3, 4, 6, 5, 3]
[2, 7, 5, 5, 1, 4, 3, 5, 4, 4, 6, 6, 4, 3, 5, 3]
[3, 4, 4, 4, 5, 7, 6, 3, 6, 5, 3, 3, 5, 6, 3, 2]
[7, 4, 2, 4, 5, 6, 2, 5, 4, 6, 2, 5, 6, 4, 5, 5]
[5, 6, 4, 3, 4, 4, 2, 6, 5, 4, 4, 6, 3, 2, 5, 7]
[4, 5, 5, 2, 5, 5, 5, 5, 3, 1, 3, 5, 4, 2, 5, 4]
[4, 5, 3, 4, 5, 5, 5, 6, 4, 6, 2, 5, 6, 3, 4, 4]
[2, 3, 5, 3, 3, 4, 4, 4, 4, 4, 2, 2, 4, 2, 2, 4]
[4, 4, 5, 3, 6, 3, 3, 2, 4, 5, 2, 2, 4, 2, 4, 2]
[6, 3, 5, 4, 4, 3, 5, 2, 5, 4, 2, 2, 2, 2, 3, 3]
[3, 3, 3, 2, 4, 3, 4, 5, 4, 4, 2, 2, 4, 2, 2, 6]
[3, 4, 2, 2, 5, 4, 3, 5, 3, 0, 5, 3, 6, 5, 3, 3]
[3, 4, 5, 3, 4, 4, 3, 3, 4, 6, 5, 2, 4, 5, 3, 3]
[3, 3, 4, 5, 3, 4, 4, 5, 3, 3, 5, 5, 6, 3, 3, 2]
[2, 4, 4, 5, 4, 3, 3, 6, 3, 5, 3, 6, 7, 5, 3, 3]
[4, 1, 6, 4, 5, 5, 4, 6, 4, 5, 5, 5, 5, 3, 6, 5]
[5, 5, 0, 4, 3, 1, 5, 3, 5, 2, 3, 3, 5, 5, 3, 4]
[4, 2, 6, 1, 5, 4, 4

In [118]:
hamming_values = []
for row in hamming_matrix:
    val = sum(row)
    hamming_values.append(val)
    print(val)

61
63
67
65
67
69
72
70
63
71
52
55
55
53
56
61
61
66
73
56
60
56
69
68
67
60
59
61
59
58
76
65
70
74
69
65
69
61
70
61
70
66
54
51
67
60
65
68
62
60
70
58
66
63
66
69
71
64
58
65
64
53
61
65
73
71
69
65
64
60
65
71
61
57
68
66
66
62
69
68
67
62
59
64
64
63
69
63
57
61
66
64
62
61
68
67
73
64
60
58
56
58
60
64
69
64
60
64
57
64
61
58
64
61
64
62
66
54
71
60
66
71
68
56
72
67
62
53
67
58
60
56
62
60
64
64
65
62
70
62
65
64
69
66
73
70
62
64
56
58
73
75
65
73
73
67
59
67
70
67
66
58
60
56
60
62
64
72
75
71
62
62
64
72
66
59
66
68
55
56
56
57
64
60
70
65
70
64
65
71
61
66
65
59
67
66
63
63
72
61
69
52
64
62
63
61
71
61
64
61
74
63
65
69
76
70
77
58
53
51
60
57
68
59
63
65
66
62
66
61
72
54
68
58
65
60
67
57
69
60
75
67
74
70
67
74
61
59
71
59
57
53
63
69
69
62


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Create histogram with bins of size 4
plt.hist(hamming_values, bins=range(32, 97, 4), edgecolor='black', density=True, alpha=0.7)

# Fit normal distribution using NumPy
mu = np.mean(hamming_values)
sigma = np.std(hamming_values)

# Generate points for normal curve
x = np.linspace(32, 97, 100)
y = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

# Plot normal curve
plt.plot(x, y, 'r-', label=f'Normal fit (μ={mu:.2f}, σ={sigma:.2f})')

# Add labels, title, and legend
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Histogram of Hamming Weights with Normal Fit')
plt.legend()

# Show plot
plt.show()

invalid command name "4862462656process_stream_events"
    while executing
"4862462656process_stream_events"
    ("after" script)


: 